In [1]:
import os
import pickle
import tensorflow as tf
import numpy as np

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
tf.__version__

'1.4.0-rc1'

In [3]:
data_directory = os.path.expanduser('~/data/')

In [4]:
imdb = tf.keras.datasets.imdb

In [5]:
imdb

<module 'tensorflow.python.keras.datasets.imdb' from '/usr/local/lib/python3.6/site-packages/tensorflow/python/keras/datasets/imdb/__init__.py'>

In [46]:
word2idx = imdb.get_word_index()

In [47]:
idx2word = { index: word for word, index in word2idx.items() }

In [48]:
len(word2idx)

88584

The dataset has 88,584 words but we will only load the top 5,000 to reduce the vocab size by not loading rare words. We will also truncate sequences to a maximum length of 500

In [262]:
VOCAB_SIZE=5_000
SEQ_LEN=500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE, maxlen=SEQ_LEN)

In [265]:
X_train_lengths = np.apply_along_axis(len, 0, X_train)

In [292]:
ds_len = lambda dataset: np.array([len(seq) for seq in dataset])
X_train_lengths = ds_len(X_train)
X_test_lengths = ds_len(X_test)

In [10]:
# One-hot style encoding
# y_train = np.array([[sentiment] for sentiment in y_train])
# y_test = np.array([[sentiment] for sentiment in y_test])

In [300]:
# Encode shape[1] as num_classes
y_train = y_train.reshape((y_train.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

In [301]:
X_train.shape, y_train.shape

((25000,), (25000, 1))

In [302]:
def pad_sequence(dataset):
    dataset = [seq + [0] * (SEQ_LEN - len(seq)) for seq in dataset]
    return np.array(dataset)

In [303]:
X_train = pad_sequence(X_train)
X_test = pad_sequence(X_test)

In [304]:
X_train.shape

(25000, 500)

### GloVe

In [15]:
EMBEDDING_DIMENSION=50 # Available dimensions for 6B data is 50, 100, 200, 300
glove_data_directory = os.path.expanduser('~/data/glove')

if not os.path.isdir(glove_data_directory):
    os.path.makedirs(glove_data_directory)
    
glove_weights_file_path = os.path.join(glove_data_directory, f'glove.6B.{EMBEDDING_DIMENSION}d.txt')

if not os.path.isfile(glove_weights_file_path):
    # Glove embedding weights can be downloaded from https://nlp.stanford.edu/projects/glove/
    glove_fallback_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    local_zip_file_path = os.path.join(glove_data_directory, os.path.basename(glove_fallback_url))
    if not os.path.isfile(local_zip_file_path):
        print(f'Retreiving glove weights from {fallback_url}')
        urllib.request.urlretrieve(glove_fallback_url, local_zip_file_path)
    with zipfile.ZipFile(local_zip_file_path, 'r') as z:
        print(f'Extracting glove weights from {local_zip_file_path}')
        z.extractall(path=data_directory)

In [49]:
gl_word2idx = {} # dict so we can lookup indices for tokenising our text later from string to sequence of integers
gl_idx2word = [] # list of words in order. Can be used to convert tokenised sequence to list of words
gl_weights = [] 
with open(glove_weights_file_path, 'r') as glove_weights:
    for index, line in enumerate(glove_weights):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
        gl_word2idx[word] = index
        gl_idx2word.append(word)
        gl_weights.append(word_weights)
        
gl_weights = np.asarray(gl_weights)
# VOCAB_LENGTH=len(gl_idx2word)
# UNKOWN_WORD=VOCAB_LENGTH-1 # Define unknown as the least frequent word we know about

In [17]:
with open(os.path.join(glove_data_directory, 'word2idx.p'), 'wb') as p:
    pickle.dump(word2idx, p)
with open(os.path.join(glove_data_directory, 'idx2word.p'), 'wb') as p:
    pickle.dump(idx2word, p)
with open(os.path.join(glove_data_directory, 'weights.npy'), 'wb') as p:
    np.save(p, weights)

In [18]:
with open(os.path.join(glove_data_directory, 'word2idx.p'), 'rb') as p:
    word2idx = pickle.load(p)
with open(os.path.join(glove_data_directory, 'idx2word.p'), 'rb') as p:
    idx2word = pickle.load(p)
with open(os.path.join(glove_data_directory, 'weights.npy'), 'rb') as p:
    weights = np.load(p)

## Estimator Infrastructure

In [194]:
def model_fn(features, labels, mode, params):
    
    logits = params['model'](features, mode, params)
    
    logit_sigmoid = tf.nn.sigmoid(logits)
    predictions = {
        'sentiment': tf.round(logit_sigmoid),
        'sentiment_rating': tf.sigmoid(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    accuracy = tf.metrics.accuracy(
        labels=labels,
        predictions=predictions['sentiment'],
        name='accuracy'
    )
    
#     loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=tf.one_hot(labels, depth=2))
    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels, logits=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss, 
            global_step=tf.train.get_global_step(), 
            learning_rate=params['learning_rate'], 
            optimizer=tf.train.AdamOptimizer)
        
        tf.summary.scalar('train_accuracy', accuracy[1])
        
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, loss=loss, train_op=train_op)
    
    eval_metrics = { 'accuracy': accuracy }
    
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, loss=loss, eval_metric_ops=eval_metrics)

In [305]:
train_batch_size = 128
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    { 'word_indices': X_train, 'seq_length': X_train_lengths },
    y=y_train,
    batch_size=train_batch_size,
    num_epochs=None, # Repeat forever
    shuffle=True
)
train_steps_per_epoch = X_train.shape[0] // train_batch_size

In [306]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    { 'word_indices': X_test, 'seq_length': X_test_lengths },
    y=y_test,
    num_epochs=1,
    shuffle=False
)

In [307]:
def get_experiment(params, max_steps=5*train_steps_per_epoch, run='1'):
    run_config = tf.contrib.learn.RunConfig()
    run_config = run_config.replace(model_dir=f'/tmp/fastai/lesson5/{params["model"].__name__}/run{run}')

    sentiment_estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params=params
    )
    
    experiment = tf.contrib.learn.Experiment(
        estimator=sentiment_estimator,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=max_steps,
        eval_delay_secs=0
    )
    
    return experiment

## Single hidden layer NN

In [308]:
def single_hidden_layer_model(features, mode, params):
    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    with tf.name_scope('embedding'):
        embedding_weights = tf.get_variable(
            name='embedding_weights',
            shape=(VOCAB_SIZE, EMBEDDING_DIMENSION))
        embedding = tf.nn.embedding_lookup(embedding_weights, features['word_indices'])
    
    with tf.name_scope('fc'):
        flat = tf.contrib.layers.flatten(embedding)
        dense_1 = tf.layers.dense(flat, units=100, activation=tf.nn.relu, trainable=is_training)
        dense_1_dropout = tf.layers.dropout(dense_1, rate=0.7, training=is_training)
        predictions = tf.layers.dense(dense_1_dropout, units=1)
        return predictions

In [309]:
single_hidden_layer_params = {
    'learning_rate': 1e-3,
    'model': single_hidden_layer_model
}

In [310]:
single_layer_experiment = get_experiment(single_hidden_layer_params)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x124f63668>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/fastai/lesson5/single_hidden_layer_model/run1'}


In [311]:
single_layer_experiment.train_and_evaluate()

INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Starting evaluation at 2017-10-30-06:29:50
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_hidden_layer_model/run1/model.ckpt-975
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:E

({'accuracy': 0.86492187, 'global_step': 975, 'loss': 0.5168345}, [])

In [221]:
single_layer_eval_metrics = single_layer_experiment.evaluate()
single_layer_eval_metrics['accuracy']

INFO:tensorflow:Starting evaluation at 2017-10-30-01:56:55
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_hidden_layer_model/run1/model.ckpt-975
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflo

0.86492187

## Single conv layer with max pooling

In [211]:
def single_conv_layer_model(features, mode, params):
    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    with tf.name_scope('embedding'):
        embedding_weights = tf.get_variable(
            name='embedding_weights', 
            shape=(VOCAB_SIZE, EMBEDDING_DIMENSION))
        embedding = tf.nn.embedding_lookup(embedding_weights, features['word_indices'])
        
    with tf.name_scope('conv'):
        dropout_1 = tf.layers.dropout(embedding, rate=0.2, training=is_training)
        conv_1 = tf.layers.conv1d(dropout_1, filters=64, kernel_size=5, padding='same', activation=tf.nn.relu)
        dropout_2 = tf.layers.dropout(conv_1, rate=0.2, training=is_training)
        max_pool = tf.layers.max_pooling1d(dropout_2, pool_size=2, strides=2)
    
    with tf.name_scope('fc'):
        flat = tf.contrib.layers.flatten(embedding)
        dense_1 = tf.layers.dense(flat, units=100, activation=tf.nn.relu, trainable=is_training)
        dense_1_dropout = tf.layers.dropout(dense_1, rate=0.7, training=is_training)
        predictions = tf.layers.dense(dense_1_dropout, units=1)
        return predictions

In [212]:
single_conv_params = {
    'learning_rate': 1e-3,
    'model': single_conv_layer_model
}

In [222]:
single_conv_layer_experiment = get_experiment(single_conv_params)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1236a6a58>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/fastai/lesson5/single_conv_layer_model/run1'}


In [214]:
single_conv_layer_experiment.train_and_evaluate()

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/fastai/lesson5/single_conv_layer_model/run1/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-10-30-01:50:14
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_conv_layer_model/run1/model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensor

INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-10-30-01:51:01
INFO:tensorflow:Saving dict for global step 975: accuracy = 0.862031, global_step = 975, loss = 0.521299


({'accuracy': 0.86203122, 'global_step': 975, 'loss': 0.5212993}, [])

In [223]:
single_conv_layer_metrics = single_conv_layer_experiment.evaluate()
single_conv_layer_metrics['accuracy']

INFO:tensorflow:Starting evaluation at 2017-10-30-01:57:25
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_conv_layer_model/run1/model.ckpt-975
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:

0.86203122

## Pre-Trained vectors

In [240]:
def create_imdb_embeddings():
    emb = np.zeros((VOCAB_SIZE, EMBEDDING_DIMENSION))
    UNKNOWN_TOKEN = len(idx2word)-1
    for i in range(1, emb.shape[0]):
        word = idx2word[i]
        glove_idx = gl_word2idx.get(word, UNKNOWN_TOKEN)
        emb[i] = gl_weights[glove_idx]
    return emb
imdb_weights = create_imdb_embeddings()
imdb_weights = imdb_weights.astype(np.float32)

In [225]:
def single_conv_layer_pretrained_model(features, mode, params):
    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    with tf.name_scope('embedding'):
        embedding_weights = tf.get_variable(
            name='embedding_weights', 
            shape=(VOCAB_SIZE, EMBEDDING_DIMENSION), 
            initializer=tf.constant_initializer(imdb_weights), # Load GLoVe weights
            trainable=params['embedding_trainable'])
        embedding = tf.nn.embedding_lookup(imdb_weights, features['word_indices'])
        
    with tf.name_scope('conv'):
        dropout_1 = tf.layers.dropout(embedding, rate=0.2, training=is_training)
        conv_1 = tf.layers.conv1d(dropout_1, filters=64, kernel_size=5, padding='same', activation=tf.nn.relu)
        dropout_2 = tf.layers.dropout(conv_1, rate=0.2, training=is_training)
        max_pool = tf.layers.max_pooling1d(dropout_2, pool_size=2, strides=2)
    
    with tf.name_scope('fc'):
        flat = tf.contrib.layers.flatten(embedding)
        dense_1 = tf.layers.dense(flat, units=100, activation=tf.nn.relu, trainable=is_training)
        dense_1_dropout = tf.layers.dropout(dense_1, rate=0.7, training=is_training)
        predictions = tf.layers.dense(dense_1_dropout, units=1)
        return predictions

In [226]:
single_conv_pretrained_params = {
    'learning_rate': 1e-3,
    'model': single_conv_layer_pretrained_model,
    'embedding_trainable': True
}

In [230]:
single_conv_layer_pretrained_experiment = get_experiment(single_conv_pretrained_params, run='2')

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x121ecc978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/fastai/lesson5/single_conv_layer_pretrained_model/run2'}


In [231]:
single_conv_layer_pretrained_experiment.train_and_evaluate()

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/fastai/lesson5/single_conv_layer_pretrained_model/run2/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-10-30-02:00:57
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_conv_layer_pretrained_model/run2/model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-10-30-02:01:49
INFO:tensorflow:Saving dict for global step 975: accuracy = 0.723125, global_step = 975, loss = 0.629988


({'accuracy': 0.72312498, 'global_step': 975, 'loss': 0.62998831}, [])

In [232]:
single_conv_layer_pretrained_eval_metrics = single_conv_layer_pretrained_experiment.evaluate()
single_conv_layer_pretrained_eval_metrics['accuracy']

INFO:tensorflow:Starting evaluation at 2017-10-30-02:01:49
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/single_conv_layer_pretrained_model/run2/model.ckpt-975
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:

0.72312498

## LSTM

In [322]:
def lstm_model(features, mode, params):
    is_training = mode==tf.estimator.ModeKeys.TRAIN
    
    with tf.name_scope('embedding'):
        embedding_weights = tf.get_variable(
            name='embedding_weights', 
            shape=(VOCAB_SIZE, EMBEDDING_DIMENSION))
        embedding = tf.nn.embedding_lookup(imdb_weights, features['word_indices'])
    
    with tf.name_scope('lstm'):
#         cell = tf.contrib.rnn.LSTMCell(num_units=100)
        cell = params['cell'](num_units=100)
        cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=0.8 if is_training else 1.0)
        outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=embedding, dtype=tf.float32, sequence_length=features['seq_length'])

        rnn_out = tf.contrib.layers.flatten(outputs)
#         rnn_out = outputs[:,-1,:]
    
    with tf.name_scope('fc'):
        predictions = tf.layers.dense(rnn_out, units=1)
        return predictions

In [323]:
lstm_params = {
    'learning_rate': 1e-3,
    'model': lstm_model,
    'cell': tf.contrib.rnn.BasicLSTMCell
}

In [324]:
lstm_experiment = get_experiment(lstm_params, max_steps=1*train_steps_per_epoch, run='7')

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1244ae2e8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/fastai/lesson5/lstm_model/run7'}


In [325]:
lstm_experiment.train_and_evaluate()

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/fastai/lesson5/lstm_model/run7/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-10-30-06:42:09
INFO:tensorflow:Restoring parameters from /tmp/fastai/lesson5/lstm_model/run7/model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
I

INFO:tensorflow:Saving dict for global step 195: accuracy = 0.599297, global_step = 195, loss = 0.664232


({'accuracy': 0.59929687, 'global_step': 195, 'loss': 0.66423172}, [])